In [1]:
import pandas as pd
import numpy as np
from datetime import timedelta
import datetime as dt
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [10, 8]
import seaborn as sns
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

# pd.options.display.max_rows = 50
pd.options.display.max_columns = 200

In [83]:
# !pip install sklearn

In [14]:
import os

file_path = "data_lagged_features.csv"
print(os.path.exists(file_path))  # Check if the file exists
print(os.access(file_path, os.R_OK))  # Check if the file is readable

True
True


In [15]:
locations = pd.read_csv("LOCATION.csv", sep=';')
products = pd.read_csv("PRODUCT.csv", sep=';')
sample = pd.read_csv("lt_sample_submission.csv", sep=',')
test_df = pd.read_csv("lt_test.csv", sep=',')
train = pd.read_csv("lt_train.csv", sep=',')
train_lag = pd.read_csv("data_lagged_features.csv", sep=',')

In [16]:
test_df

,product_id,location_id,period_dt,id
0,23297,317,2019-01-21,601
1,23297,644,2019-01-28,697
2,23297,1162,2019-01-14,831
3,23297,1316,2019-02-25,875
4,23301,644,2019-01-28,975
...,...,...,...,...
10164,135868,1316,2019-11-11,408050
10165,135869,1316,2019-11-11,408051
10166,149418,1316,2019-11-11,408052
10167,40608,453,2019-11-25,408053


In [17]:
train_lag

,Unnamed: 0,period_dt,location_id,product_id,demand,PROMO1_FLAG,PROMO2_FLAG,PRICE_REGULAR,PRICE_AFTER_DISC,NUM_CONSULTANT,AUTORIZATION_FLAG,id,weekday,monthday,is_weekend,NoFilter,ind_of_year,ind_of_month,ind_of_day,holiday_flag,holiday_flag2,demand_lag22d_alpha14_keyproduct_id_preagsum_PROMO1_FLAG_dynamic_ewm,demand_lag22d_alpha28_keyproduct_id_preagsum_PROMO1_FLAG_dynamic_ewm,demand_lag22d_w14D_keyproduct_id_preagsum_agmean_PROMO1_FLAG_dynamic_rolling,demand_lag22d_w14D_keyproduct_id_preagsum_agpctl10_PROMO1_FLAG_dynamic_rolling,demand_lag22d_w14D_keyproduct_id_preagsum_agpctl90_PROMO1_FLAG_dynamic_rolling,demand_lag22d_w21D_keyproduct_id_preagsum_agmean_PROMO1_FLAG_dynamic_rolling,demand_lag22d_w21D_keyproduct_id_preagsum_agpctl10_PROMO1_FLAG_dynamic_rolling,demand_lag22d_w21D_keyproduct_id_preagsum_agpctl90_PROMO1_FLAG_dynamic_rolling,demand_lag22d_w28D_keyproduct_id_preagsum_agmean_PROMO1_FLAG_dynamic_rolling,demand_lag22d_w28D_keyproduct_id_preagsum_agpctl10_PROMO1_FLAG_dynamic_rolling,demand_lag22d_w28D_keyproduct_id_preagsum_agpctl90_PROMO1_FLAG_dynamic_rolling,demand_lag22d_w56D_keyproduct_id_preagsum_agmean_PROMO1_FLAG_dynamic_rolling,demand_lag22d_w56D_keyproduct_id_preagsum_agpctl10_PROMO1_FLAG_dynamic_rolling,demand_lag22d_w56D_keyproduct_id_preagsum_agpctl90_PROMO1_FLAG_dynamic_rolling,demand_lag28d_alpha14_keyproduct_id_preagsum_PROMO1_FLAG_dynamic_ewm,demand_lag28d_alpha28_keyproduct_id_preagsum_PROMO1_FLAG_dynamic_ewm,demand_lag28d_w14D_keyproduct_id_preagsum_agmean_PROMO1_FLAG_dynamic_rolling,demand_lag28d_w14D_keyproduct_id_preagsum_agpctl10_PROMO1_FLAG_dynamic_rolling,demand_lag28d_w14D_keyproduct_id_preagsum_agpctl90_PROMO1_FLAG_dynamic_rolling,demand_lag28d_w21D_keyproduct_id_preagsum_agmean_PROMO1_FLAG_dynamic_rolling,demand_lag28d_w21D_keyproduct_id_preagsum_agpctl10_PROMO1_FLAG_dynamic_rolling,demand_lag28d_w21D_keyproduct_id_preagsum_agpctl90_PROMO1_FLAG_dynamic_rolling,demand_lag28d_w28D_keyproduct_id_preagsum_agmean_PROMO1_FLAG_dynamic_rolling,demand_lag28d_w28D_keyproduct_id_preagsum_agpctl10_PROMO1_FLAG_dynamic_rolling,demand_lag28d_w28D_keyproduct_id_preagsum_agpctl90_PROMO1_FLAG_dynamic_rolling,demand_lag28d_w56D_keyproduct_id_preagsum_agmean_PROMO1_FLAG_dynamic_rolling,demand_lag28d_w56D_keyproduct_id_preagsum_agpctl10_PROMO1_FLAG_dynamic_rolling,demand_lag28d_w56D_keyproduct_id_preagsum_agpctl90_PROMO1_FLAG_dynamic_rolling,demand_lag35d_alpha14_keyproduct_id_preagsum_PROMO1_FLAG_dynamic_ewm,demand_lag35d_alpha28_keyproduct_id_preagsum_PROMO1_FLAG_dynamic_ewm,demand_lag35d_w14D_keyproduct_id_preagsum_agmean_PROMO1_FLAG_dynamic_rolling,demand_lag35d_w14D_keyproduct_id_preagsum_agpctl10_PROMO1_FLAG_dynamic_rolling,demand_lag35d_w14D_keyproduct_id_preagsum_agpctl90_PROMO1_FLAG_dynamic_rolling,demand_lag35d_w21D_keyproduct_id_preagsum_agmean_PROMO1_FLAG_dynamic_rolling,demand_lag35d_w21D_keyproduct_id_preagsum_agpctl10_PROMO1_FLAG_dynamic_rolling,demand_lag35d_w21D_keyproduct_id_preagsum_agpctl90_PROMO1_FLAG_dynamic_rolling,demand_lag35d_w28D_keyproduct_id_preagsum_agmean_PROMO1_FLAG_dynamic_rolling,demand_lag35d_w28D_keyproduct_id_preagsum_agpctl10_PROMO1_FLAG_dynamic_rolling,demand_lag35d_w28D_keyproduct_id_preagsum_agpctl90_PROMO1_FLAG_dynamic_rolling,demand_lag35d_w56D_keyproduct_id_preagsum_agmean_PROMO1_FLAG_dynamic_rolling,demand_lag35d_w56D_keyproduct_id_preagsum_agpctl10_PROMO1_FLAG_dynamic_rolling,demand_lag35d_w56D_keyproduct_id_preagsum_agpctl90_PROMO1_FLAG_dynamic_rolling,demand_lag22d_alpha14_keyproduct_id_preagsum_NoFilter_dynamic_ewm,demand_lag22d_alpha28_keyproduct_id_preagsum_NoFilter_dynamic_ewm,demand_lag22d_w14D_keyproduct_id_preagsum_agmean_NoFilter_dynamic_rolling,demand_lag22d_w14D_keyproduct_id_preagsum_agpctl10_NoFilter_dynamic_rolling,demand_lag22d_w14D_keyproduct_id_preagsum_agpctl90_NoFilter_dynamic_rolling,demand_lag22d_w21D_keyproduct_id_preagsum_agmean_NoFilter_dynamic_rolling,demand_lag22d_w21D_keyproduct_id_preagsum_agpctl10_NoFilter_dynamic_rolling,demand_lag22d_w

In [18]:
unique_counts = train.nunique()
zero_counts = (train == 0).sum()
nan_counts = train.isnull().sum()

total_rows = len(train)
zero_percentage = zero_counts / total_rows * 100
nan_percentage = nan_counts / total_rows * 100

In [19]:
pd.options.display.max_rows = 110

summary_stats = pd.DataFrame({
    'Уникальные значения': unique_counts,
    'Нулевые значения': zero_counts,
    'Доля нулевых значений %': zero_percentage,
    'Пустые значения': nan_counts,
    'Доля пустых значений %': nan_percentage,
    'Тип данных': train.dtypes
})

summary_stats

,Уникальные значения,Нулевые значения,Доля нулевых значений %,Пустые значения,Доля пустых значений %,Тип данных
period_dt,159,0,0.000000,0,0.000000,object
location_id,41,0,0.000000,3001944,88.166807,float64
product_id,2821,0,0.000000,3001944,88.166807,float64
demand,14335,3888,0.114190,3155253,92.669478,float64
PROMO1_FLAG,4,2039259,59.892841,2504,0.073542,float64
PROMO2_FLAG,2,3396807,99.763895,2504,0.073542,float64
PRICE_REGULAR,2236,0,0.000000,1222255,35.897512,float64
PRICE_AFTER_DISC,12278,0,0.000000,1257110,36.921200,float64
NUM_CONSULTANT,11,3310066,97.216321,2504,0.073542,float64
AUTORIZATION_FLAG,2,777713,22.841356,2504,0.073542,float64


In [20]:
unique_counts = train_lag.nunique()
zero_counts = (train_lag == 0).sum()
nan_counts = train_lag.isnull().sum()

total_rows = len(train_lag)
zero_percentage = zero_counts / total_rows * 100
nan_percentage = nan_counts / total_rows * 100

In [21]:
pd.options.display.max_rows = 110

summary_stats = pd.DataFrame({
    'Уникальные значения': unique_counts,
    'Нулевые значения': zero_counts,
    'Доля нулевых значений %': zero_percentage,
    'Пустые значения': nan_counts,
    'Доля пустых значений %': nan_percentage,
    'Тип данных': train_lag.dtypes
})

summary_stats

,Уникальные значения,Нулевые значения,Доля нулевых значений %,Пустые значения,Доля пустых значений %,Тип данных
Unnamed: 0,3404846,1,0.000029,0,0.000000,int64
period_dt,159,0,0.000000,0,0.000000,object
location_id,41,0,0.000000,3001944,88.166807,float64
product_id,2821,0,0.000000,3001944,88.166807,float64
demand,14335,3888,0.114190,3155253,92.669478,float64
PROMO1_FLAG,4,2041763,59.966383,0,0.000000,float64
PROMO2_FLAG,2,3399311,99.837438,0,0.000000,float64
PRICE_REGULAR,2236,0,0.000000,0,0.000000,float64
PRICE_AFTER_DISC,12441,0,0.000000,0,0.000000,float64
NUM_CONSULTANT,11,3312565,97.289716,0,0.000000,float64


In [22]:
pd.options.display.max_rows = 10

In [23]:
new_test = train_lag[train_lag['id'].isin(test_df['id'])]

In [24]:
train_lag = train_lag[~train_lag['id'].isin(test_df['id'])]

In [25]:
train_lag = train_lag.dropna(subset=['demand'])

In [26]:
new_test

,Unnamed: 0,period_dt,location_id,product_id,demand,PROMO1_FLAG,PROMO2_FLAG,PRICE_REGULAR,PRICE_AFTER_DISC,NUM_CONSULTANT,AUTORIZATION_FLAG,id,weekday,monthday,is_weekend,NoFilter,ind_of_year,ind_of_month,ind_of_day,holiday_flag,holiday_flag2,demand_lag22d_alpha14_keyproduct_id_preagsum_PROMO1_FLAG_dynamic_ewm,demand_lag22d_alpha28_keyproduct_id_preagsum_PROMO1_FLAG_dynamic_ewm,demand_lag22d_w14D_keyproduct_id_preagsum_agmean_PROMO1_FLAG_dynamic_rolling,demand_lag22d_w14D_keyproduct_id_preagsum_agpctl10_PROMO1_FLAG_dynamic_rolling,demand_lag22d_w14D_keyproduct_id_preagsum_agpctl90_PROMO1_FLAG_dynamic_rolling,demand_lag22d_w21D_keyproduct_id_preagsum_agmean_PROMO1_FLAG_dynamic_rolling,demand_lag22d_w21D_keyproduct_id_preagsum_agpctl10_PROMO1_FLAG_dynamic_rolling,demand_lag22d_w21D_keyproduct_id_preagsum_agpctl90_PROMO1_FLAG_dynamic_rolling,demand_lag22d_w28D_keyproduct_id_preagsum_agmean_PROMO1_FLAG_dynamic_rolling,demand_lag22d_w28D_keyproduct_id_preagsum_agpctl10_PROMO1_FLAG_dynamic_rolling,demand_lag22d_w28D_keyproduct_id_preagsum_agpctl90_PROMO1_FLAG_dynamic_rolling,demand_lag22d_w56D_keyproduct_id_preagsum_agmean_PROMO1_FLAG_dynamic_rolling,demand_lag22d_w56D_keyproduct_id_preagsum_agpctl10_PROMO1_FLAG_dynamic_rolling,demand_lag22d_w56D_keyproduct_id_preagsum_agpctl90_PROMO1_FLAG_dynamic_rolling,demand_lag28d_alpha14_keyproduct_id_preagsum_PROMO1_FLAG_dynamic_ewm,demand_lag28d_alpha28_keyproduct_id_preagsum_PROMO1_FLAG_dynamic_ewm,demand_lag28d_w14D_keyproduct_id_preagsum_agmean_PROMO1_FLAG_dynamic_rolling,demand_lag28d_w14D_keyproduct_id_preagsum_agpctl10_PROMO1_FLAG_dynamic_rolling,demand_lag28d_w14D_keyproduct_id_preagsum_agpctl90_PROMO1_FLAG_dynamic_rolling,demand_lag28d_w21D_keyproduct_id_preagsum_agmean_PROMO1_FLAG_dynamic_rolling,demand_lag28d_w21D_keyproduct_id_preagsum_agpctl10_PROMO1_FLAG_dynamic_rolling,demand_lag28d_w21D_keyproduct_id_preagsum_agpctl90_PROMO1_FLAG_dynamic_rolling,demand_lag28d_w28D_keyproduct_id_preagsum_agmean_PROMO1_FLAG_dynamic_rolling,demand_lag28d_w28D_keyproduct_id_preagsum_agpctl10_PROMO1_FLAG_dynamic_rolling,demand_lag28d_w28D_keyproduct_id_preagsum_agpctl90_PROMO1_FLAG_dynamic_rolling,demand_lag28d_w56D_keyproduct_id_preagsum_agmean_PROMO1_FLAG_dynamic_rolling,demand_lag28d_w56D_keyproduct_id_preagsum_agpctl10_PROMO1_FLAG_dynamic_rolling,demand_lag28d_w56D_keyproduct_id_preagsum_agpctl90_PROMO1_FLAG_dynamic_rolling,demand_lag35d_alpha14_keyproduct_id_preagsum_PROMO1_FLAG_dynamic_ewm,demand_lag35d_alpha28_keyproduct_id_preagsum_PROMO1_FLAG_dynamic_ewm,demand_lag35d_w14D_keyproduct_id_preagsum_agmean_PROMO1_FLAG_dynamic_rolling,demand_lag35d_w14D_keyproduct_id_preagsum_agpctl10_PROMO1_FLAG_dynamic_rolling,demand_lag35d_w14D_keyproduct_id_preagsum_agpctl90_PROMO1_FLAG_dynamic_rolling,demand_lag35d_w21D_keyproduct_id_preagsum_agmean_PROMO1_FLAG_dynamic_rolling,demand_lag35d_w21D_keyproduct_id_preagsum_agpctl10_PROMO1_FLAG_dynamic_rolling,demand_lag35d_w21D_keyproduct_id_preagsum_agpctl90_PROMO1_FLAG_dynamic_rolling,demand_lag35d_w28D_keyproduct_id_preagsum_agmean_PROMO1_FLAG_dynamic_rolling,demand_lag35d_w28D_keyproduct_id_preagsum_agpctl10_PROMO1_FLAG_dynamic_rolling,demand_lag35d_w28D_keyproduct_id_preagsum_agpctl90_PROMO1_FLAG_dynamic_rolling,demand_lag35d_w56D_keyproduct_id_preagsum_agmean_PROMO1_FLAG_dynamic_rolling,demand_lag35d_w56D_keyproduct_id_preagsum_agpctl10_PROMO1_FLAG_dynamic_rolling,demand_lag35d_w56D_keyproduct_id_preagsum_agpctl90_PROMO1_FLAG_dynamic_rolling,demand_lag22d_alpha14_keyproduct_id_preagsum_NoFilter_dynamic_ewm,demand_lag22d_alpha28_keyproduct_id_preagsum_NoFilter_dynamic_ewm,demand_lag22d_w14D_keyproduct_id_preagsum_agmean_NoFilter_dynamic_rolling,demand_lag22d_w14D_keyproduct_id_preagsum_agpctl10_NoFilter_dynamic_rolling,demand_lag22d_w14D_keyproduct_id_preagsum_agpctl90_NoFilter_dynamic_rolling,demand_lag22d_w21D_keyproduct_id_preagsum_agmean_NoFilter_dynamic_rolling,demand_lag22d_w21D_keyproduct_id_preagsum_agpctl10_NoFilter_dynamic_rolling,demand_lag22d_w

In [27]:
null_metric = train_lag.isnull().sum(axis=1)
null_table = null_metric.value_counts().sort_index()

null_table

16       1
24      56
27       6
32    4013
33      28
      ... 
69    1039
70     291
71     310
72    4115
84      16
Name: count, Length: 45, dtype: int64

In [28]:
train_lag['period_dt'] = pd.to_datetime(train_lag['period_dt'])
first_arrival = train_lag.groupby(['product_id', 'location_id'])['period_dt'].min().reset_index()
first_arrival.rename(columns={'period_dt': 'arrival'}, inplace=True)
train_lag = train_lag.merge(first_arrival, on=['product_id', 'location_id'])

In [29]:
train_lag['period_dt'] = pd.to_datetime(train_lag['period_dt'])
train_lag['arrival'] = pd.to_datetime(train_lag['arrival'])

train_lag['days_diff'] = (train_lag['period_dt'] - train_lag['arrival']).dt.days
train_lag['is2w'] = train_lag['days_diff'].apply(lambda x: 1 if x <= 14 else 0)
train_lag.drop(columns=['days_diff'], inplace=True)

In [30]:
new_train = train_lag[train_lag['is2w'] != 0]

In [31]:
new_train

,Unnamed: 0,period_dt,location_id,product_id,demand,PROMO1_FLAG,PROMO2_FLAG,PRICE_REGULAR,PRICE_AFTER_DISC,NUM_CONSULTANT,AUTORIZATION_FLAG,id,weekday,monthday,is_weekend,NoFilter,ind_of_year,ind_of_month,ind_of_day,holiday_flag,holiday_flag2,demand_lag22d_alpha14_keyproduct_id_preagsum_PROMO1_FLAG_dynamic_ewm,demand_lag22d_alpha28_keyproduct_id_preagsum_PROMO1_FLAG_dynamic_ewm,demand_lag22d_w14D_keyproduct_id_preagsum_agmean_PROMO1_FLAG_dynamic_rolling,demand_lag22d_w14D_keyproduct_id_preagsum_agpctl10_PROMO1_FLAG_dynamic_rolling,demand_lag22d_w14D_keyproduct_id_preagsum_agpctl90_PROMO1_FLAG_dynamic_rolling,demand_lag22d_w21D_keyproduct_id_preagsum_agmean_PROMO1_FLAG_dynamic_rolling,demand_lag22d_w21D_keyproduct_id_preagsum_agpctl10_PROMO1_FLAG_dynamic_rolling,demand_lag22d_w21D_keyproduct_id_preagsum_agpctl90_PROMO1_FLAG_dynamic_rolling,demand_lag22d_w28D_keyproduct_id_preagsum_agmean_PROMO1_FLAG_dynamic_rolling,demand_lag22d_w28D_keyproduct_id_preagsum_agpctl10_PROMO1_FLAG_dynamic_rolling,demand_lag22d_w28D_keyproduct_id_preagsum_agpctl90_PROMO1_FLAG_dynamic_rolling,demand_lag22d_w56D_keyproduct_id_preagsum_agmean_PROMO1_FLAG_dynamic_rolling,demand_lag22d_w56D_keyproduct_id_preagsum_agpctl10_PROMO1_FLAG_dynamic_rolling,demand_lag22d_w56D_keyproduct_id_preagsum_agpctl90_PROMO1_FLAG_dynamic_rolling,demand_lag28d_alpha14_keyproduct_id_preagsum_PROMO1_FLAG_dynamic_ewm,demand_lag28d_alpha28_keyproduct_id_preagsum_PROMO1_FLAG_dynamic_ewm,demand_lag28d_w14D_keyproduct_id_preagsum_agmean_PROMO1_FLAG_dynamic_rolling,demand_lag28d_w14D_keyproduct_id_preagsum_agpctl10_PROMO1_FLAG_dynamic_rolling,demand_lag28d_w14D_keyproduct_id_preagsum_agpctl90_PROMO1_FLAG_dynamic_rolling,demand_lag28d_w21D_keyproduct_id_preagsum_agmean_PROMO1_FLAG_dynamic_rolling,demand_lag28d_w21D_keyproduct_id_preagsum_agpctl10_PROMO1_FLAG_dynamic_rolling,demand_lag28d_w21D_keyproduct_id_preagsum_agpctl90_PROMO1_FLAG_dynamic_rolling,demand_lag28d_w28D_keyproduct_id_preagsum_agmean_PROMO1_FLAG_dynamic_rolling,demand_lag28d_w28D_keyproduct_id_preagsum_agpctl10_PROMO1_FLAG_dynamic_rolling,demand_lag28d_w28D_keyproduct_id_preagsum_agpctl90_PROMO1_FLAG_dynamic_rolling,demand_lag28d_w56D_keyproduct_id_preagsum_agmean_PROMO1_FLAG_dynamic_rolling,demand_lag28d_w56D_keyproduct_id_preagsum_agpctl10_PROMO1_FLAG_dynamic_rolling,demand_lag28d_w56D_keyproduct_id_preagsum_agpctl90_PROMO1_FLAG_dynamic_rolling,demand_lag35d_alpha14_keyproduct_id_preagsum_PROMO1_FLAG_dynamic_ewm,demand_lag35d_alpha28_keyproduct_id_preagsum_PROMO1_FLAG_dynamic_ewm,demand_lag35d_w14D_keyproduct_id_preagsum_agmean_PROMO1_FLAG_dynamic_rolling,demand_lag35d_w14D_keyproduct_id_preagsum_agpctl10_PROMO1_FLAG_dynamic_rolling,demand_lag35d_w14D_keyproduct_id_preagsum_agpctl90_PROMO1_FLAG_dynamic_rolling,demand_lag35d_w21D_keyproduct_id_preagsum_agmean_PROMO1_FLAG_dynamic_rolling,demand_lag35d_w21D_keyproduct_id_preagsum_agpctl10_PROMO1_FLAG_dynamic_rolling,demand_lag35d_w21D_keyproduct_id_preagsum_agpctl90_PROMO1_FLAG_dynamic_rolling,demand_lag35d_w28D_keyproduct_id_preagsum_agmean_PROMO1_FLAG_dynamic_rolling,demand_lag35d_w28D_keyproduct_id_preagsum_agpctl10_PROMO1_FLAG_dynamic_rolling,demand_lag35d_w28D_keyproduct_id_preagsum_agpctl90_PROMO1_FLAG_dynamic_rolling,demand_lag35d_w56D_keyproduct_id_preagsum_agmean_PROMO1_FLAG_dynamic_rolling,demand_lag35d_w56D_keyproduct_id_preagsum_agpctl10_PROMO1_FLAG_dynamic_rolling,demand_lag35d_w56D_keyproduct_id_preagsum_agpctl90_PROMO1_FLAG_dynamic_rolling,demand_lag22d_alpha14_keyproduct_id_preagsum_NoFilter_dynamic_ewm,demand_lag22d_alpha28_keyproduct_id_preagsum_NoFilter_dynamic_ewm,demand_lag22d_w14D_keyproduct_id_preagsum_agmean_NoFilter_dynamic_rolling,demand_lag22d_w14D_keyproduct_id_preagsum_agpctl10_NoFilter_dynamic_rolling,demand_lag22d_w14D_keyproduct_id_preagsum_agpctl90_NoFilter_dynamic_rolling,demand_lag22d_w21D_keyproduct_id_preagsum_agmean_NoFilter_dynamic_rolling,demand_lag22d_w21D_keyproduct_id_preagsum_agpctl10_NoFilter_dynamic_rolling,demand_lag22d_w

In [32]:
new_train.drop(columns=['Unnamed: 0'], inplace=True)

C:\Users\user\AppData\Local\Temp\ipykernel_29580\2426005829.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_train.drop(columns=['Unnamed: 0'], inplace=True)


In [33]:
null_metric = new_train.isnull().sum(axis=1)
null_table = null_metric.value_counts().sort_index()

null_table

24       1
32    1443
33      11
34      21
35     409
      ... 
69     487
70     112
71     138
72    2157
84      14
Name: count, Length: 43, dtype: int64

In [34]:
columns_to_drop = [col for col in new_train.columns if 'demand_lag' in col]
new_train.drop(columns=columns_to_drop, inplace=True)

C:\Users\user\AppData\Local\Temp\ipykernel_29580\2696278067.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_train.drop(columns=columns_to_drop, inplace=True)


In [35]:
new_train

,period_dt,location_id,product_id,demand,PROMO1_FLAG,PROMO2_FLAG,PRICE_REGULAR,PRICE_AFTER_DISC,NUM_CONSULTANT,AUTORIZATION_FLAG,id,weekday,monthday,is_weekend,NoFilter,ind_of_year,ind_of_month,ind_of_day,holiday_flag,holiday_flag2,arrival,is2w
0,2016-12-19,309.0,23252.0,0.388889,0.0,0.0,2099.0,2099.0,0.0,1.0,277726,0,19,0,1,2016,12,19,0,0,2016-12-19,1
1,2016-12-19,1203.0,77298.0,2.250000,0.0,0.0,2599.0,2599.0,0.0,1.0,104336,0,19,0,1,2016,12,19,0,0,2016-12-19,1
2,2016-12-19,355.0,80753.0,1.000000,0.0,0.0,1199.0,1199.0,0.0,1.0,137327,0,19,0,1,2016,12,19,0,0,2016-12-19,1
3,2016-12-19,1203.0,77300.0,2.250000,0.0,0.0,2399.0,2399.0,0.0,1.0,104926,0,19,0,1,2016,12,19,0,0,2016-12-19,1
4,2016-12-19,1079.0,35384.0,0.500000,0.0,0.0,2549.0,2549.0,0.0,1.0,40085,0,19,0,1,2016,12,19,0,0,2016-12-19,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
249564,2018-12-24,1326.0,115240.0,1.000000,1.0,0.0,2299.0,1149.5,0.0,0.0,259960,0,24,0,1,2018,12,24,0,0,2018-12-24,1
249572,2018-12-24,862.0,74471.0,1.000000,0.0,0.0,866.0,866.0,0.0,0.0,76717,0,24,0,1,2018,12,24,0,0,2018-12-24,1
249585,2018-12-24,1326.0,101460.0,1.000000,1.0,0.0,2199.0,1099.5,6.0,0.0,244248,0,24,0,1,2018,12,24,0,0,2018-12-24,1
249590,2018-12-24,1326.0,101456.0,1.000000,1.0,0.0,2199.0,1099.5,6.0,0.0,243806,0,24,0,1,2018,12,24,0,0,2018-12-24,1


In [36]:
null_metric = new_train.isnull().sum(axis=1)
null_table = null_metric.value_counts().sort_index()

null_table

0    42277
Name: count, dtype: int64

In [37]:
nan_counts = new_train.isnull().sum()

summary_stats_missings = pd.DataFrame({
    'Пустые значения': nan_counts,
    'Тип данных': new_train.dtypes
})

summary_stats_missings

,Пустые значения,Тип данных
period_dt,0,datetime64[ns]
location_id,0,float64
product_id,0,float64
demand,0,float64
PROMO1_FLAG,0,float64
...,...,...
ind_of_day,0,int64
holiday_flag,0,int64
holiday_flag2,0,int64
arrival,0,datetime64[ns]


In [54]:
X = new_train.drop(['id', 'demand'], axis=1)
y = new_train['demand']

In [55]:
for col in X.select_dtypes(include=['datetime64']).columns:
    X[col] = X[col].view(int)

C:\Users\user\AppData\Local\Temp\ipykernel_29580\1245744823.py:2: FutureWarning: Series.view is deprecated and will be removed in a future version. Use ``astype`` as an alternative to change the dtype.
  X[col] = X[col].view(int)


In [97]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [98]:
features_to_learn = X_train.columns
features_to_learn = features_to_learn.drop(['PRICE_AFTER_DISC', 'AUTORIZATION_FLAG', 'weekday', 'NUM_CONSULTANT', 'is_weekend', 'period_dt', 'NoFilter', 'arrival', 'is2w'])
features_to_learn

Index(['location_id', 'product_id', 'PROMO1_FLAG', 'PROMO2_FLAG',
       'PRICE_REGULAR', 'monthday', 'ind_of_year', 'ind_of_month',
       'ind_of_day', 'holiday_flag', 'holiday_flag2'],
      dtype='object')

In [99]:
X_train = X_train[features_to_learn]
X_test = X_test[features_to_learn]

In [46]:
# !pip install xgboost

In [100]:
from sklearn.metrics import mean_squared_error
from sklego.meta import ZeroInflatedRegressor
from sklearn.linear_model import LogisticRegression
import xgboost as xgb

In [101]:
classifier = LogisticRegression()

xgb_regressor = xgb.XGBRegressor(random_state=1)

zir = ZeroInflatedRegressor(classifier=classifier, regressor=xgb_regressor)

zir.fit(X_train, y_train)

y_pred = zir.predict(X_test)

rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(f'Root Mean Squared Error: {rmse}')

c:\Users\user\miniconda3\lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Root Mean Squared Error: 0.7671982763157588


In [102]:
from sklearn.metrics import mean_absolute_error

mae = mean_absolute_error(y_test, y_pred)
print(f'Mean Absolute Error: {mae}')

Mean Absolute Error: 0.4748527613355517


In [67]:
new_test.drop(columns=['Unnamed: 0'], inplace=True)

C:\Users\user\AppData\Local\Temp\ipykernel_29580\1990158751.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_test.drop(columns=['Unnamed: 0'], inplace=True)


In [70]:
columns_to_drop = [col for col in new_test.columns if 'demand_lag' in col]
new_test.drop(columns=columns_to_drop, inplace=True)

C:\Users\user\AppData\Local\Temp\ipykernel_29580\2086883045.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_test.drop(columns=columns_to_drop, inplace=True)


In [103]:
new_test

,period_dt,location_id,product_id,demand,PROMO1_FLAG,PROMO2_FLAG,PRICE_REGULAR,PRICE_AFTER_DISC,NUM_CONSULTANT,AUTORIZATION_FLAG,id,weekday,monthday,is_weekend,NoFilter,ind_of_year,ind_of_month,ind_of_day,holiday_flag,holiday_flag2
1780105,2019-01-07,1363.0,111860.0,NaN,0.0,0.0,3267.0,2205.23,0.0,0.0,258373,0,7,0,1,2019,1,7,0,0
1781066,2019-01-07,1363.0,107571.0,NaN,0.0,0.0,2697.0,2022.75,0.0,0.0,253137,0,7,0,1,2019,1,7,0,0
1781551,2019-01-07,1326.0,88960.0,NaN,0.0,0.0,2049.0,1024.50,0.0,1.0,397128,0,7,0,1,2019,1,7,0,0
1782088,2019-01-07,453.0,77302.0,NaN,0.0,0.0,2499.0,1686.83,0.0,1.0,105433,0,7,0,1,2019,1,7,0,0
1782252,2019-01-07,1326.0,81473.0,NaN,1.0,0.0,2437.0,1218.50,0.0,1.0,140963,0,7,0,1,2019,1,7,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3395603,2019-12-30,557.0,88910.0,NaN,1.0,0.0,3149.0,944.70,0.0,1.0,179895,0,30,0,1,2019,12,30,0,0
3395676,2019-12-30,644.0,81848.0,NaN,0.0,0.0,2949.0,884.70,0.0,1.0,148035,0,30,0,1,2019,12,30,0,0
3395685,2019-12-30,644.0,140916.0,NaN,1.0,0.0,2549.0,1274.50,0.0,1.0,272154,0,30,0,1,2019,12,30,0,0
3395866,2019-12-30,637.0,119468.0,NaN,1.0,0.0,2749.0,1374.50,0.0,1.0,264120,0,30,0,1,2019,12,30,0,0


In [72]:
sample

,id,demand
0,601,1.0
1,697,1.0
2,831,1.0
3,875,1.0
4,975,1.0
...,...,...
10164,408050,1.0
10165,408051,1.0
10166,408052,1.0
10167,408053,1.0


In [104]:
X_test = new_test.drop(['id', 'demand'], axis=1)
y_pred_res = zir.predict(X_test[features_to_learn])
y_results = new_test[['id', 'demand']]
y_results['demand'] = y_pred_res
y_results.loc[y_results['demand'] < 0, ['demand']] = 0.0
y_results.loc[y_results['demand'] < 0, ['demand']]

C:\Users\user\AppData\Local\Temp\ipykernel_29580\3584685674.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y_results['demand'] = y_pred_res


,demand


In [105]:
y_results = y_results.sort_values(by='id')

In [106]:
y_results.to_csv('initial_subbmit.csv', sep=',', encoding='utf-8', index=False)

In [107]:
y_results

,id,demand
1844440,601,0.884725
1874310,697,0.936299
1827718,831,1.451133
1957238,875,1.413401
1874309,975,0.910110
...,...,...
3150602,408050,0.000000
3150603,408051,0.000000
3150613,408052,0.000000
3221065,408053,1.011539


In [77]:
sample

,id,demand
0,601,1.0
1,697,1.0
2,831,1.0
3,875,1.0
4,975,1.0
...,...,...
10164,408050,1.0
10165,408051,1.0
10166,408052,1.0
10167,408053,1.0
